# An introduction to Models in Pyro

https://pyro.ai/examples/intro_part_i.html

In [1]:
import torch
import pyro

pyro.set_rng_seed(101)

## Primitive Stochastic Functions 

In [2]:
# 標準正規分布の作成

loc = 0 # mean zero
scale = 1 # unit variance
normal = torch.distributions.Normal(loc, scale) #標準正規分布のオブジェクト作成

x = normal.rsample() # 上のオブジェクトの分布に沿ってランダムにサンプルを生成(tensor形式)

print("sample", x)
print("log prob", normal.log_prob(x)) #score the sample from N(0,1)

sample tensor(-1.3905)
log prob tensor(-1.8857)


## A Simple Model
天気と気温の関係をモデル化する。

+ 使う確率分布：ベルヌーイ分布、正規分布
+ 曇りの確率：30%
+ 気温の平均値 = 曇り：55度、晴れ：75度 (華氏)
+ 気温の標準偏差 = 曇り:10度、晴れ：15度

戻り値：天気('cloudy' or 'sunny'), 気温(float)

In [3]:
def weather_torch():
    cloudy = torch.distributions.Bernoulli(0.3).sample()
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy':10.0, 'sunny': 75.0}[cloudy]
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()
    return cloudy, temp.item()

## The Pyro.sample Primitive

ここまでの内容をtorchではなく、pyroの記法で書き直す。例えばx=normal.rsample()が、下記のように変わる

x = pyro.sample('名前', pyro.distributions.Normal(loc, scale)

この名前を基に、pyroのバックエンドではサンプルの識別を行い、各サンプルの挙動の変更を行っていく

In [4]:
x = pyro.sample("my_sample", pyro.distributions.Normal(loc, scale))
print(x)

tensor(-0.8152)


In [5]:
def weather():
    cloudy = pyro.sample('cloudy', pyro.distributions.Bernoulli(0.3))
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy':10.0, 'sunny': 75.0}[cloudy]
    temp = pyro.sample('temp', pyro.distributions.Normal(mean_temp, scale_temp))
    return cloudy, temp.item()

for _ in range(3):
    print(weather())

('cloudy', 64.5440444946289)
('sunny', 171.8779296875)
('sunny', 62.59336853027344)


## Universality: Stochastic Recursion, Higher-order Stochastic Functions, and Random Control Flow

例として、ice_cleam_sales()を作成する。天気や気温によって売り上げは変わるが、それをweather関数とつなげてシンプルに記載できる

+ 売上期待値 = 80度以上で晴れの日:200.0, その他: 50.0
+ 売上標準偏差 = 10.0


In [6]:
def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = 200.0 if ('sunny' and temp > 80.0) else 50.0
    ice_cream = pyro.sample('ice_cream', pyro.distributions.Normal(expected_sales, 10.0))
    return ice_cream

再帰的な関数も生成することができる。例として、幾何分布を作成する

In [7]:
def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), pyro.distributions.Bernoulli(p))
    if x.item() == 1:
        return 0
    else:
        return 1 + geometric(p, t + 1)

print(geometric(0.1)) #公式ページ例では初回で1が出て再帰していないため、確率pを下げ再帰が正常に起きていることを確認

12


同様に、独自の確率関数を定義することもできる。

In [8]:
def normal_product(loc, scale):
    z1 = pyro.sample("z1", pyro.distributions.Normal(loc, scale))
    z2 = pyro.sample("z2", pyro.distributions.Normal(loc, scale))
    y = z1 * z2
    return y

def make_normal_normal():
    mu_latent = pyro.sample("mu_latent", pyro.distributions.Normal(0, 1))
    fn = lambda scale: normal_product(mu_latent, scale)
    return fn

print(make_normal_normal()(1.0))

tensor(-1.4257)
